# Exemple d'extraction de graphe

## Setup

***Load modules*** 
This will require amtrack repo in the right path I use environment variables but feel free to do something better.
Normally all the required packages are indicated, if some are missing, please add them.
If you also want to create a "freeze" or a poetry thing to have an easier installation of the environment please do so.


In [22]:
%load_ext autoreload
%autoreload 2
import os  
import sys  
import pandas as pd
from amftrack.util.sys import get_dates_datetime, get_dirname, temp_path, get_data_info, update_plate_info, \
get_current_folders, get_folders_by_plate_id, fiji_path, path_code
import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.image_processing.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.image_processing.experiment_class_surf import Experiment, Edge,Node, plot_raw_plus
from amftrack.pipeline.paths.directory import run_parallel_transfer, find_state, directory_scratch, directory_project, directory_archive
import dropbox
from amftrack.transfer.functions.transfer import upload, zip_file
from subprocess import call
from tqdm.autonotebook import tqdm
from time import time_ns
import subprocess
import matplotlib as mpl
from amftrack.util.sys import temp_path

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


***choose the path of interest***
I mounted sun on my linux computer at AMOLF. Please feel free to do so on yours as well. I can send you the command. 
Otherwise, if you have the folders locally, just indicate the path.
Finally you can also simply ssh to my computer and run things from there.

In [12]:
directory = "/data/felix/width1/full_plates/" # careful: must have the / at the end

update_plate_info(directory)
folder_df = get_current_folders(directory)

analysed:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
len(folder_df)

2

***Choose the folder you're interested in***

In [14]:
folder_df

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/skeleton_pruned_compressed.mat,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,Temp
12704,/data/felix/width1/full_plates/20220330_0049_P...,865,7,'Carrot','A5','001P100N100C','Y','-',20220307,25,...,False,False,False,False,False,False,False,"30.03.2022, 00:49:",20220330_0049_Plate943,NaN
12705,/data/felix/width1/full_plates/20220325_1423_P...,907,9,'Carrot','A5','001P100N100C','Y','-',20220321,25,...,False,False,False,False,False,False,False,"25.03.2022, 14:23:",20220325_1423_Plate907,NaN


In [ ]:
str(folder_df['total_path'].iloc[1])

In [15]:
folders = folder_df.loc[folder_df['Plate']=="907"]

In [16]:
folders

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/skeleton_pruned_compressed.mat,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,Temp
12705,/data/felix/width1/full_plates/20220325_1423_P...,907,9,'Carrot','A5','001P100N100C','Y','-',20220321,25,...,False,False,False,False,False,False,False,"25.03.2022, 14:23:",20220325_1423_Plate907,NaN


These are local adaptations of functions for running things on the supercomputer. So better to keep them in the notebook at this stage.

## Stiching function

In [23]:
from amftrack.util.sys import path_code
import os

def make_stitching_loop(directory,dirname,op_id):
    # TODO(FK): handle the case where there is no stiching_loops directory
    #MODIF a_file = open(f'{path_code}amftrack/pipeline/scripts/stitching_loops/stitching_loop.ijm',"r")
    a_file = open(os.path.join(path_code, "pipeline/scripts/stitching_loops/stitching_loop.ijm") ,"r")
    
    list_of_lines = a_file.readlines()

    list_of_lines[4] = f'mainDirectory = \u0022{directory}\u0022 ;\n'
    list_of_lines[29] = f'\t if(startsWith(list[i],\u0022{dirname}\u0022)) \u007b\n'
    file_name = f'{os.getenv("TEMP")}/stitching_loops/stitching_loop{op_id}.ijm'
    a_file = open(file_name, "w")

    a_file.writelines(list_of_lines)

    a_file.close()
    
def run_stitch(directory, folders):
    folder_list = list(folders['folder'])
    folder_list.sort()    
    with tqdm(total=len(folder_list), desc="stitched") as pbar:
        for folder in folder_list:
            op_id = time_ns()
            im = imageio.imread(f'{directory}/{folder}/Img/Img_r03_c05.tif')
            for x in range(1,11):
                for y in range(1,16):
                    strix= str(x) if x>=10 else f'0{x}'
                    striy= str(y) if y>=10 else f'0{y}'
                    path = f'{directory}/{folder}/Img/Img_r{strix}_c{striy}.tif'
                    # print(striy,path,os.path.getsize(path))
                    if not os.path.isfile(path):
                        f = open(path, 'w')
                    if os.path.getsize(path)==0:
                        imageio.imwrite(path,im*0)
            make_stitching_loop(directory,folder,op_id)
            # MODIF 2 command = [os.getenv('HOME')+'/Fiji.app/ImageJ-linux64','--mem=8000m','--headless','--ij2','--console','-macro',f'{os.getenv("TEMP")}/stitching_loops/stitching_loop{op_id}.ijm']
            command = [fiji_path,'--mem=8000m','--headless','--ij2','--console','-macro',f'{os.getenv("TEMP")}/stitching_loops/stitching_loop{op_id}.ijm']
            print(' '.join(command))
            process = subprocess.run(command)
            pbar.update(1)

In [9]:
def run_parallel(code, args, folders):
    op_id = time_ns()
    folders.to_json(f'{os.getenv("TEMP")}/{op_id}.json')# temporary file
    folder_list = list(folders['folder'])
    folder_list.sort()  
    args_str = [str(arg) for arg in args]
    arg_str = " ".join(args_str)
    with tqdm(total=len(folder_list), desc="stitched") as pbar:
        for index,folder in enumerate(folder_list):
            command = ['python3',f'{path_code}pipeline/scripts/image_processing/{code}']+args_str+[f'{op_id}',f'{index}']
            print(' '.join(command))
            process = subprocess.run(command)
            pbar.update(1)

***Stitch the folders you're interested in***

should take about 20mins

In [24]:
run_stitch(directory,folders)

stitched:   0%|          | 0/1 [00:00<?, ?it/s]

/home/ipausers/kahane/opt/fiji-linux64/Fiji.app/ImageJ-linux64 --mem=8000m --headless --ij2 --console -macro /data/felix/TEMP/stitching_loops/stitching_loop1649863777621808632.ijm


Java HotSpot(TM) 64-Bit Server VM warning: ignoring option PermSize=128m; support was removed in 8.0
Java HotSpot(TM) 64-Bit Server VM warning: Using incremental CMS is deprecated and will likely be removed in a future release


20220325_1423_Plate907/
(Wed Apr 13 17:32:01 CEST 2022): Stitching the following files:
Image: '/data/felix/width1/full_plates//20220325_1423_Plate907//Img2/Img_r01_c01.tif' Imp: 'null' Offset: (0.0, 0.0)
Image: '/data/felix/width1/full_plates//20220325_1423_Plate907//Img2/Img_r01_c02.tif' Imp: 'null' Offset: (3276.0, 0.0)
Image: '/data/felix/width1/full_plates//20220325_1423_Plate907//Img2/Img_r01_c03.tif' Imp: 'null' Offset: (6552.0, 0.0)
Image: '/data/felix/width1/full_plates//20220325_1423_Plate907//Img2/Img_r01_c04.tif' Imp: 'null' Offset: (9828.0, 0.0)
Image: '/data/felix/width1/full_plates//20220325_1423_Plate907//Img2/Img_r01_c05.tif' Imp: 'null' Offset: (13104.0, 0.0)
Image: '/data/felix/width1/full_plates//20220325_1423_Plate907//Img2/Img_r01_c06.tif' Imp: 'null' Offset: (16380.0, 0.0)
Image: '/data/felix/width1/full_plates//20220325_1423_Plate907//Img2/Img_r01_c07.tif' Imp: 'null' Offset: (19656.0, 0.0)
Image: '/data/felix/width1/full_plates//20220325_1423_Plate907//Img2/Img

0 overlaps 1: 0.0 translation: x: 2156 y: 23
0 overlaps 15: 0.0 translation: x: -5280 y: 2509
0 overlaps 16: 0.0 translation: x: 2156 y: 2505
1 overlaps 2: 0.0 translation: x: 2156 y: 23
1 overlaps 15: 0.0 translation: x: -4396 y: 2505
1 overlaps 16: 0.0 translation: x: -5280 y: 2509
1 overlaps 17: 0.0 translation: x: 2156 y: 2505
2 overlaps 3: 0.0 translation: x: 2156 y: 23
2 overlaps 16: 0.0 translation: x: -4396 y: 2505
2 overlaps 17: 0.0 translation: x: -5280 y: 2509
2 overlaps 18: 0.0 translation: x: 2156 y: 2505
3 overlaps 4: 0.0 translation: x: 2510 y: 742
3 overlaps 17: 0.0 translation: x: -4396 y: 2505
3 overlaps 18: 0.0 translation: x: -4453 y: 2609
3 overlaps 19: 0.0 translation: x: 2391 y: 2622
4 overlaps 5: 0.5898469877476745 translation: x: 3369 y: -11
4 overlaps 18: 0.2296289209886288 translation: x: -3289 y: 2461
4 overlaps 19: 0.09145192564422139 translation: x: 754 y: 2380
4 overlaps 20: 0.2881825737165599 translation: x: 3377 y: 2050
5 overlaps 6: 0.0597991293948763 

...

72 overlaps 87: 0.7723523612774651 translation: x: -15 y: 2556
72 overlaps 88: 0.4213256248074557 translation: x: 3416 y: 2548
73 overlaps 74: 0.0 translation: x: 2364 y: -909
73 overlaps 87: 0.45873256435419285 translation: x: -3444 y: 2571
73 overlaps 88: 0.7599684008053751 translation: x: -13 y: 2565
73 overlaps 89: 0.7417155402815098 translation: x: 3432 y: 2557
74 overlaps 88: 0.0 translation: x: -2397 y: 2700
74 overlaps 89: 0.0 translation: x: 4703 y: 2107
75 overlaps 76: 0.9436132190920686 translation: x: 3346 y: -11
75 overlaps 90: 0.7109296675561765 translation: x: -2 y: 2577
75 overlaps 91: 0.2863434829016504 translation: x: 3339 y: 2577
76 overlaps 77: 0.9136137910396241 translation: x: 3442 y: -14
76 overlaps 90: 0.17941874117940146 translation: x: -3396 y: 2419
76 overlaps 91: 0.561802167509126 translation: x: -27 y: 2572
76 overlaps 92: 0.05134486834946514 translation: x: 3055 y: 2331
77 overlaps 78: 0.6794593207309881 translation: x: 3436 y: -15
77 overlaps 91: 0.170927

107 overlaps 108: 0.7692833191546301 translation: x: 3436 y: -12
107 overlaps 121: 0.08354169956045696 translation: x: -3419 y: 2540
107 overlaps 122: 0.4602255631327187 translation: x: -8 y: 2555
107 overlaps 123: 0.42442544424387774 translation: x: 3455 y: 2546
108 overlaps 109: 0.08643383348195839 translation: x: 3439 y: -28
108 overlaps 122: 0.0 translation: x: -2204 y: 1625
108 overlaps 123: 0.3434898155826444 translation: x: -6 y: 2560
108 overlaps 124: 0.013785308685334561 translation: x: 3401 y: 2904
109 overlaps 110: 0.557990961231044 translation: x: 3439 y: -10
109 overlaps 123: 0.06756112453865733 translation: x: -3969 y: 2144
109 overlaps 124: 0.4734865940325984 translation: x: -6 y: 2554
109 overlaps 125: 0.5561973027543403 translation: x: 3434 y: 2536
110 overlaps 111: 0.6515649021316957 translation: x: 3436 y: -14
110 overlaps 124: 0.5345780527518356 translation: x: -3447 y: 2564
110 overlaps 125: 0.7497582039468695 translation: x: -6 y: 2552
110 overlaps 126: 0.38299605

Successfully optimized configuration of 128 tiles after 10000 iterations:
  average displacement: 40.479px
  minimal displacement: 0.000px
  maximal displacement: 485.832px
maxError more than 6.0 times bigger than avgerror.
Identified link between /data/felix/width1/full_plates//20220325_1423_Plate907//Img2/Img_r01_c10.tif and /data/felix/width1/full_plates//20220325_1423_Plate907//Img2/Img_r01_c11.tif (R=0.1709631443872382) to be bad. Reoptimizing.
Tile size: 128
Number of tile graphs = 1
Successfully optimized configuration of 128 tiles after 10000 iterations:
  average displacement: 33.905px
  minimal displacement: 0.000px
  maximal displacement: 236.022px
maxError more than 6.0 times bigger than avgerror.
Identified link between /data/felix/width1/full_plates//20220325_1423_Plate907//Img2/Img_r06_c03.tif and /data/felix/width1/full_plates//20220325_1423_Plate907//Img2/Img_r07_c02.tif (R=0.17092724769731318) to be bad. Reoptimizing.
Tile size: 128
Number of tile graphs = 1
Successfu

Successfully optimized configuration of 128 tiles after 5764 iterations:
  average displacement: 8.306px
  minimal displacement: 0.000px
  maximal displacement: 45.772px
maxError more than 6.0 times bigger than avgerror.
Identified link between /data/felix/width1/full_plates//20220325_1423_Plate907//Img2/Img_r09_c05.tif and /data/felix/width1/full_plates//20220325_1423_Plate907//Img2/Img_r10_c05.tif (R=0.1933641048696744) to be bad. Reoptimizing.
Tile size: 128
Number of tile graphs = 1
Successfully optimized configuration of 128 tiles after 5764 iterations:
  average displacement: 7.739px
  minimal displacement: 0.000px
  maximal displacement: 45.761px
maxError more than 6.0 times bigger than avgerror.
Identified link between /data/felix/width1/full_plates//20220325_1423_Plate907//Img2/Img_r08_c14.tif and /data/felix/width1/full_plates//20220325_1423_Plate907//Img2/Img_r09_c15.tif (R=0.14601483500137474) to be bad. Reoptimizing.
Tile size: 128
Number of tile graphs = 1
Successfully op

(Wed Apr 13 17:43:34 CEST 2022): Finished Stitching.
Saving /data/felix/width1/full_plates//20220325_1423_Plate907//Stitched Image_1.tif ... 
Saved
Image grid sequence stitching is done!
1


***Compress the image and skeletonize***

should take 1h30-2h

In [25]:
args = [directory]
run_parallel('compress_image.py',args, folders)

stitched:   0%|          | 0/1 [00:00<?, ?it/s]

python3 /home/ipausers/kahane/Wks/AMFtrack/amftrack/pipeline/scripts/image_processing/compress_image.py /data/felix/width1/full_plates/ 1649864771751109735 0
Directory  /data/felix/width1/full_plates/20220325_1423_Plate907/Analysis  Created 


In [ ]:
time = '3:00:00'
low = 30
high = 80
extend = 30
args=[low, high, extend,directory]
run_parallel('extract_skel.py',args,folders)

## Visualisation

***Visualise the skeletonization***

In [ ]:
%matplotlib inline
import mpld3
mpld3.enable_notebook()

In [ ]:
select = folders
folder_list = list(select['total_path'])
folder_list.sort()
skels = []
ims = []
kernel = np.ones((5,5),np.uint8)
itera = 1
start = 0

finish = start+2
for folder in folder_list[start:finish]:
    path_snap=folder
    skel_info = read_mat(path_snap+'/Analysis/skeleton_compressed.mat')
    skel = skel_info['skeleton']
    skels.append(cv2.dilate(skel.astype(np.uint8),kernel,iterations = itera))
    im = read_mat(path_snap+'/Analysis/raw_image.mat')['raw']
    ims.append(im)

In [ ]:
plt.close('all')
start = 0
finish = start + 1
for i in range(start,finish):
    plot_t_tp1([], [], None, None, skels[i], ims[i])

***These steps may be a bit long, but it also helps just keeping things consistent and use one single framework,*** you may bypass them by artificially renaming skeleton.mat into skeleton_realigned.mat which is necessary for the next step because you probably don't care about these two steps since you're doing only one image...

In [ ]:
time = '2:00'
thresh = 50
args=[thresh, directory]
run_parallel('mask_skel.py',args,folders)

In [ ]:
time = '3:00:00'
threshold = 0.1
args = [threshold, directory]
run_parallel('prune_skel.py',args,folders)

In [ ]:
plates = list(set(list(folders['Plate'].values)))
for plate in plates:
    time = '2:00:00'
    args = [directory]
    run_parallel('realign.py', args, folders)

Now you're finally getting to the point where you generate the graph object

In [ ]:
time = '2:00:00'
args=[directory]
run_parallel('extract_nx_graph.py',args, folders)

***and now you have the Experiment object to play with and query images related to position of edges etc...***


In [ ]:
run_info = folders
plate = int(list(run_info['folder'])[i].split('_')[-1][5:])
folder_list = list(run_info['folder'])
directory_name = folder_list[i]
exp = Experiment(plate,directory)
exp.load(run_info.loc[run_info['folder']==directory_name],labeled=False)
path_snap = directory + directory_name
(G, pos) = exp.nx_graph[0],exp.positions[0]

In [ ]:
from random import choice
edge = choice(list(G.edges))

In [ ]:
edge_exp = Edge(Node(edge[0],exp),Node(edge[1],exp),exp)

***Visualiser l'image correspondant à l'edge*** 

***Visualiser tout le réseau avec l'edge idniqué dessus***

## Brouillon

In [ ]:
exp

In [ ]:
dir(exp)